# An interactive phase plane

To try it without having to download the repository to: 
https://colab.research.google.com/drive/164Gzja9miNliiXIscJxG23ysRlxVgde-


In [5]:
import path_setup
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy as dcp 
from WC_lib.model_utils import *
from WC_lib.stability_analysis_utils import *
from ipywidgets import interactive, fixed, FloatText, Text, IntText,Layout, Box

In [ ]:
# General plotting settings
vector_field_resolution = 25
vector_field_arrow_width = 0.0005
dpi = 110
image_scaling = 1.5

In [6]:
def plot_phase_plane(
                        bifurcate, ncols,
                        c_ee, c_ei, c_ie, c_ii,
                        theta_e, theta_i,
                        a_e, a_i,
                        tau_e,tau_i,
                        state_space_range,
                        plot_bounds_range, figargs
    ):
    # Read the plot settings from the text boxes according to their format
    bifurcation_settings = [bif.split(",") for bif in bifurcate.split(" | ")]
    state_space_str = state_space_range.split(",")
    plot_bounds = [float(pb) for pb in plot_bounds_range.split(",")]
    state_space = [[float(state_space_str[0]),float(state_space_str[1])],[float(state_space_str[2]),float(state_space_str[3])]]
    # Define the space for nullclines
    Es, Is = get_Es_Is_log_edges(state_space,figargs["n_points"], edge_res=-3)
    # Define the parameter set of interest
    parameters_default = np.array([
                                c_ee, c_ei, c_ie, c_ii,                 #c_xx
                                tau_e,tau_i ,                           #tau_x
                                theta_e, theta_i,                       #theta_x
                                a_e, a_i,                               #a_x
                                1.0, 1.0, 1.0, 1.0,                     #k_x, r_x
                                0, 0                                    #noise
                    ])
    # Plotting settings
    subplot_size = figargs["subplot_size"]
    nrows = np.ceil(len(bifurcation_settings)/ncols).astype(int)
    fig = plt.figure(figsize = (subplot_size[0]*ncols,subplot_size[1]*nrows), dpi = dpi)
    # For each subplot
    for i in range(len(bifurcation_settings)):
        ax = plt.subplot(nrows,ncols,1+i)
        # Find if the nullcline that changes is E_null or I_null
        bifpar = bifurcation_settings[i][0]
        if bifpar in ["c_ee", "c_ei","theta_e", "a_e"]:
            iterate_over_I = False
        elif bifpar in ["c_ie", "c_ii","theta_i", "a_i"]:
            iterate_over_I = True
        # Define the parameter values for the given range
        bif_space = np.linspace(float(bifurcation_settings[i][1]),float(bifurcation_settings[i][2]),int(bifurcation_settings[i][3]))
        # Format the parameters
        parameters_set = np.zeros((bif_space.shape[0],parameters_default.shape[0])) + parameters_default[None,:]
        parameters_set[:,parameter_index_mapping[bifpar]] = bif_space
        # Define the color gradient
        ax.set_prop_cycle('color', plt.cm.viridis(np.linspace(0,1,bif_space.shape[0])))
        # Plot each nullcline
        for p, parameters in enumerate(parameters_set):
            Enull = [Is,fn_E_nullcline(Is, parameters,0)]
            Inull = [fn_I_nullcline(Es, parameters,0),Es]
            if iterate_over_I:
                plt.plot(Enull[0],Enull[1], lw = 0.75, label=f"{round(bif_space[p],2)}")
            else:
                plt.plot(Inull[0],Inull[1], lw = 0.75, label=f"{round(bif_space[p],2)}")
        if len(parameters_set)<12: # Plot the legend (12 points would make the legend too big)
            plt.legend(ncols=min(len(parameters_set),3),columnspacing=0.5)
        
        # Plot the nullclines for the parameter set of interest (parameters_default)
        Enull = [Is,fn_E_nullcline(Is, parameters_default,0)]
        Inull = [fn_I_nullcline(Es, parameters_default,0),Es]
        plt.plot(Inull[0],Inull[1], "--g", lw = 1.75)
        plt.plot(Enull[0],Enull[1], "--r", lw = 1.75)
        # Plot the associated vector field 
        Es_quiver = np.linspace(plot_bounds[2],plot_bounds[3],vector_field_resolution)
        Is_quiver = np.linspace(plot_bounds[0],plot_bounds[1],vector_field_resolution)
        Edot_grid = np.zeros((Es_quiver.shape[0],Is_quiver.shape[0]))
        Idot_grid = np.zeros((Es_quiver.shape[0],Is_quiver.shape[0]))
        for j in range(Es_quiver.shape[0]):
            Edot_grid[j] = fn_Edot(Es_quiver[j],Is_quiver,parameters_default,0)
            Idot_grid[j] = fn_Idot(Es_quiver[j],Is_quiver,parameters_default)
        xgrid, ygrid = np.meshgrid(Is_quiver,Es_quiver)
        flow = plt.quiver(xgrid,ygrid,Idot_grid,Edot_grid, width = vector_field_arrow_width, angles="xy")
        # Define the plot bounds
        plt.xlim(plot_bounds[0]-0.0125,plot_bounds[1]+0.0125)
        plt.ylim(plot_bounds[2]-0.0125,plot_bounds[3]+0.0125)
        plt.xlabel("Inhibitory")
        plt.ylabel("Excitatory")
        # Set the title of the subplot
        plt.title(parameter_latex_mapping[bifpar])
    plt.tight_layout()
    plt.show()

def get_widget_dict(
                    parameters_interest,
                    bif_pars, n_bif_pts=5,
                    width_init = 0.25,
                    init_ncols = 4,
                    
        ):
    init_bif = " | ".join([",".join([f"{bif_pars[p]}",f"{round(parameters_interest[parameter_index_mapping[bif_pars[p]]]*(1-width_init),3)}",f"{round(parameters_interest[parameter_index_mapping[bif_pars[p]]]*(1+width_init),3)}",str(n_bif_pts)]) for p in range(len(bif_pars))])
    widget_dict = dict(
                bifurcate = Text(
                    value=init_bif,
                    placeholder='Bif_par0,start,end,nstep | Bif_par1,start,end,nstep|...',
                    description='bifurcation:',
                    continuous_update=False,
                    disabled=False,
                    layout = Layout(width='76.2525%', height="40px")
                ),
                c_ee = FloatText(
                            value=parameters_interest[0],
                            description='c_ee',
                            disabled=False,continuous_update=False
                        ),
                c_ei = FloatText(
                            value=parameters_interest[1],
                            description='c_ei',
                            disabled=False,continuous_update=False
                        ),
                c_ie = FloatText(
                            value=parameters_interest[2],
                            description='c_ie',
                            disabled=False,continuous_update=False
                        ),
                c_ii = FloatText(
                            value=parameters_interest[3],
                            description='c_ii',
                            disabled=False,continuous_update=False
                        ),
                theta_e = FloatText(
                            value=parameters_interest[6],
                            description='theta_e',
                            disabled=False,continuous_update=False
                        ),
                theta_i = FloatText(
                            value=parameters_interest[7],
                            description='theta_i',
                            disabled=False,continuous_update=False
                        ),
                a_e = FloatText(
                            value=parameters_interest[8],
                            description='a_e',
                            disabled=False,continuous_update=False
                        ),
                a_i = FloatText(
                            value=parameters_interest[9],
                            description='a_i',
                            disabled=False,continuous_update=False
                        ),
                tau_e = FloatText(
                            value=parameters_interest[4],
                            description='tau_e',
                            disabled=False,continuous_update=False
                        ),
                tau_i = FloatText(
                            value=parameters_interest[5],
                            description='tau_i',
                            disabled=False,continuous_update=False
                        ),
                state_space_range = Text(
                    value='0,0.5,0,0.5',
                    placeholder='I_min,I_max,E_min,E_max',
                    description='State Space:',
                    disabled=False,continuous_update=False
                ),
                plot_bounds_range = Text(
                    value='0,0.5,0,0.5',
                    placeholder='I_min,I_max,E_min,E_max',
                    description='Plot space:',
                    disabled=False,continuous_update=False
                ),
                ncols = IntText(
                            value=init_ncols,
                            description='ncols',
                            disabled=False,continuous_update=False
                        ),
        )
    return widget_dict

# What the interactive plot can do

The interactive plot displays the phase plane for a set parameter values (with the nullclines shown as dashed lines) which can be changed using the text boxes that are associated with each parameter. 

Additionally, it can take in a list of parameters to explore within a certain range with a certain number of points through the "bifurcation" text box. 

For each parameter in this list, it will make a copy of the phase plane and add the nullclines for each parameter value listed within the range defined for this parameter, with the nullclines following a color gradient.

This allows the exploration of changes in parameter values around a certain parameter set of interest for multiple parameters at the same time.

## Interface explanation

Editing the code for "parameters_interest", "bif_pars", and "widget_dict", will change the initial state of the interactive plot, but will not restrain what can be done with it as all features are controllable through the interactive text boxes.

To define the parameter set of interest, simply edit the value associated with each parameter's respective box. 

To define the exploration around this parameter set of interest simply edit the text in the "bifurcation" text box respecting the following format: parameter_0,start_value,end_value,number_of_points | parameter_1,start_value,end_value,number_of_points | ...

Some additional inputs are defined such as "ncols", which allows control over the number of plots per row (if looking at multiple parameters at once), "State Space" which defines where the nullclines are to be located and "Plot space" which defines what limits are for the plots (allows you to zoom-in around some region).

Note that there is only 1 vector field shown in the plot, even though multiple nullclines are present. This vector field is associated with the parameter set of interest (the dashed nullclines) and it is not changed by what is in the "bifurcation" text box.

### Exploring a single parameter

In [7]:
parameters_interest = limit_cycle40Hz
bif_pars = ["c_ee"]
widget_dict = get_widget_dict(
                    parameters_interest,
                    bif_pars, n_bif_pts=8,
                    width_init = 0.1,
                    init_ncols = 1,
                    
        )
figargs = {
    "subplot_size":(4*image_scaling,3.75*image_scaling),
    "n_points":100, # Defines the number of points used to plot the nullclines.
}
interactive_plot = interactive(plot_phase_plane, figargs=fixed(figargs),**widget_dict)
output = interactive_plot.children[-1]
widget_interactives = interactive_plot.children[:-1]
display(Box(widget_interactives[:len(widget_interactives)], layout = Layout(
                                                                                flex_flow='row wrap',
                                                                                width='100%', 
                                                                                align_items='flex-start',
                                                                                justify_content = "flex-start"
                    )))
display((output))

Box(children=(Text(value='c_ee,14.4,17.6,8', continuous_update=False, description='bifurcation:', layout=Layou…

Output()

### Exploring multiple parameters simultaneously

Note that the vector field and the dashed lines are the same for each subplot, only the explored nullclines are changed across subplots based on the parameter range that it is showing.

If the plots are too small you can change the number of columns "ncols" or change the "subplot_size" item in the "figargs" dict. If the text in the plots is too small you can change the "dpi" variable in the "plot_phase_plane" function defined in the 2nd cell.

In [9]:
parameters_interest = limit_cycle40Hz
bif_pars = ["c_ee","c_ei","theta_e","a_e","c_ie","c_ii","theta_i","a_i"]
widget_dict = get_widget_dict(
                    parameters_interest,
                    bif_pars, n_bif_pts=8,
                    width_init = 0.25,
                    init_ncols = 3,
                    
        )
figargs = {
    "subplot_size":(4*image_scaling,3.75*image_scaling),
    "n_points":100, # Defines the number of points used to plot the nullclines.
}
interactive_plot = interactive(plot_phase_plane, figargs=fixed(figargs),**widget_dict)
output = interactive_plot.children[-1]
widget_interactives = interactive_plot.children[:-1]
display(Box(widget_interactives[:len(widget_interactives)], layout = Layout(
                                                                                flex_flow='row wrap',
                                                                                width='100%', 
                                                                                align_items='flex-start',
                                                                                justify_content = "flex-start"
                    )))
display((output))

Box(children=(Text(value='c_ee,12.0,20.0,8 | c_ei,9.0,15.0,8 | theta_e,1.5,2.5,8 | a_e,0.975,1.625,8 | c_ie,11…

Output()